In [ ]:
import json


def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


def parse_json(json_string):
    json_dict = json.loads(json_string)
    return json_dict


def prettify_json(obj):
    pretty_json = json.dumps(obj, indent=2)
    return pretty_json


def write_to_json_file(obj, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(obj, json_file, indent=2)


In [ ]:
import configparser

def read_ini_file(file_path):
    config = configparser.ConfigParser()
    config.read(file_path)
    ini_dict = {section: dict(config.items(section))
                for section in config.sections()}
    return ini_dict


In [ ]:
def transform_graph(graph):
	nodes = { node['data']['id']: node['data'] for node in graph['elements']['nodes'] }
	edges = {}
	for edge in graph['elements']['edges']:
		if 'label' in edge['data']:
			label = edge['data']['label']
		else:
			label = ','.join(edge['data']['labels'])
			edge['data']['label'] = label
		
		if label not in edges:
			edges[label] = []
		edges[label].append(edge['data'])
	return (nodes, edges)
	

In [ ]:
config = read_ini_file('config.ini')
project_name = config['project']['name']
project_desc = config['project']['desc']
ifile = config['project']['ifile']
(project_name,project_desc,ifile)

In [ ]:
graph = read_json_file(ifile)
nodes,edges = transform_graph(graph)

list(nodes.keys()), list(edges.keys())

In [ ]:

node_header = ("id:ID",":LABEL","fullName","simpleName","color","dependencyProfileCategory","cohesion")
edge_header = ("id",":TYPE",":START_ID",":END_ID","references","dependencyTypes","nrDependencies:INT","nrCalls:INT")

In [47]:
domain_name = f'{project_name}-project'
domain_node = (domain_name,"Domain",domain_name,domain_name,"#666666",None,None)
app_name = f'{project_name}-app'
app_node = (app_name,"Component",app_name,app_name,"#666666",None,None)

contains = [(f"{domain_name}-contains-{app_name}","CONTAINS",domain_name,app_name,"{}",None,None,None)]


In [ ]:
nodes,edges = transform_graph(graph)

In [31]:
list(nodes.keys())

['weka.core.expressionlanguage.common.SimpleVariableDeclarations',
 'weka.attributeSelection.CfsSubsetEval',
 'weka.core.stopwords.Null',
 'weka.gui.boundaryvisualizer.BoundaryVisualizer',
 'weka.gui.streams.SerialInstanceListener',
 'weka.gui.beans.KnowledgeFlowApp$InvisibleTreeModel',
 'weka.core.pmml.jaxbbindings.Discretize',
 'weka.core.expressionlanguage.parser.Scanner',
 'weka.gui.beans.DataSourceListener',
 'weka.gui.beans.GraphEvent',
 'weka.gui.PackageManager$Progressable',
 'weka.gui.boundaryvisualizer.RemoteBoundaryVisualizerSubTask',
 'weka.classifiers.trees.j48.NBTreeModelSelection',
 'weka.core.pmml.jaxbbindings.Chebychev',
 'weka.gui.JTableHelper',
 'weka.gui.HierarchyPropertyParser',
 'weka.core.expressionlanguage.common.Operators',
 'weka.gui.beans.MetaBean',
 'weka.core.expressionlanguage.common.JavaMacro$InvalidSignature',
 'weka.estimators.UnivariateNormalEstimator',
 'weka.gui.explorer.ExplorerDefaults',
 'weka.core.neighboursearch.balltrees.MiddleOutConstructor$Li

In [32]:
list(edges.keys())

['contains',
 'specializes',
 'nests',
 'calls',
 'constructs',
 'holds',
 'accepts',
 'returns']

In [ ]:
# ("id:ID",":LABEL","fullName","simpleName","color","dependencyProfileCategory","cohesion")

# packages that directly contain classes
sublayers = list(set((edge['source'],'Sublayer',edge['source'],nodes[edge['source']]['properties']['simpleName'],'#666666',None,None) 
	for edge in edges['contains'] if 'Container' in nodes[edge['source']]['labels'] and 'Structure' in nodes[edge['target']]['labels']))

sublayers

In [ ]:
# ("id",":TYPE",":START_ID",":END_ID","references","dependencyTypes","nrDependencies:INT","nrCalls:INT")
contains += [(f'{app_name}-contains-{id}','CONTAINS',app_name,id,"{}",None,None,None) for id,label,fullName,simpleName,color,depProfileCat,cohesion in sublayers]

In [ ]:
# ("id",":TYPE",":START_ID",":END_ID","references","dependencyTypes","nrDependencies:INT","nrCalls:INT")
contains += [(edge['id'],'CONTAINS',edge['source'],edge['target'],"{}",None,None,None)
	for edge in edges['contains'] if 'Container' in nodes[edge['source']]['labels'] and 'Structure' in nodes[edge['target']]['labels']]

contains

In [ ]:
# ("id",":TYPE",":START_ID",":END_ID","references","dependencyTypes","nrDependencies:INT","nrCalls:INT")
calls = [(f'{edge["source"]}-calls-{edge["target"]}','CALLS',edge['source'],edge['target'],"{}","compile_time",edge['properties']['weight'],None)
	for edge in edges['calls'] if edge['source'] != edge['target']]

calls

In [ ]:
def invert(edgeList):
    prefix = "inv_"
    invertedEdges = []
    for edge in edgeList:
        invertedEdge = {
            'source': edge['target'],
            'target': edge['source'],
            'label': prefix + edge.get('label', ''),
            **{key: value for key, value in edge.items() if key not in ['source', 'target', 'label']}
        }
        invertedEdges.append(invertedEdge)
    return invertedEdges

parents = {e['source']:e['target'] for e in invert(edges['contains'])}
parents

In [ ]:
from collections import Counter

dependencyProfiles = dict()

for edge in edges['calls']:

	srcId = edge['source']
	tgtId = edge['target']

	if not srcId in dependencyProfiles:
		dependencyProfiles[srcId] = list()
	if not tgtId in dependencyProfiles:
		dependencyProfiles[tgtId] = list()
	
	if parents[srcId] != parents[tgtId]:
		dependencyProfiles[srcId].append('out')
		dependencyProfiles[tgtId].append('in')

dependencyProfiles = {id:Counter(profile) for id, profile in dependencyProfiles.items()}

In [ ]:
def depProfile(inn,out):
	if inn==0 and out>0:
		return "outbound"
	elif inn>0 and out==0:
		return "inbound"
	elif inn>0 and out>0:
		return "transit"
	else:
		return "hidden"

In [ ]:
dependencyProfiles = {id:depProfile(profile['in'],profile['out']) 
		for id, profile in dependencyProfiles.items()}

In [ ]:
dependencyProfiles

In [ ]:

projProfile = Counter(dependencyProfiles.values())
projProfile

In [ ]:
l = (projProfile['hidden'],projProfile['inbound'],projProfile['outbound'],projProfile['transit'])
n = sum(l)
tuple(i/n for i in l)

In [ ]:
# ("id:ID",":LABEL","fullName","simpleName","color","dependencyProfileCategory","cohesion")
modules = [(id,'Module',id,node['properties']['simpleName'],'#cccccc',dependencyProfiles.get(id,None),None)
	for id,node in nodes.items() if 'Structure' in node['labels'] and id != 'java.lang.String']

modules

In [ ]:
import csv

with open(f"{project_name}-nodes.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(node_header)
    writer.writerow(domain_node)
    writer.writerow(app_node)
    writer.writerows(sublayers)
    writer.writerows(modules)

with open(f"{project_name}-edges.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(edge_header)
    writer.writerows(contains)
    writer.writerows(calls)